In [1]:
from __future__ import print_function
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

In [2]:
import tensorflow as tf
import keras

/home/harshit/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
from keras.applications import vgg16
from keras.layers import Input
from keras import backend as k
from keras.preprocessing.image import load_img,img_to_array

In [4]:
k.set_image_dim_ordering('tf')
image_path= '/home/harshit/Downloads/image.jpg'
result_prefix = '/home/harshit/Downloads'

height = 750
width = 1200

In [5]:
settings_preset= {
    'dreamy' : {
        'features' : {
            'block5_conv1': 0.05,
            'block5_conv2': 0.02
        },
        'continuity': 0.1,
        'jitter': 0.0,
        'dreaml2': 0.02
    }
}

In [6]:
settings= settings_preset['dreamy']

In [7]:
def preprocess_image(image_path):
    img= load_img(image_path, target_size=(height,width))
    img= img_to_array(img)
    img= np.expand_dims(img,axis=0)
    img= vgg16.preprocess_input(img)
    return img

In [8]:
def deprocess_image(x):
    x=x.reshape((height,width,3))
    x[:,:,0]+= 103.939
    x[:,:,1]+= 116.779
    x[:,:,2]+= 123.680
    
    x=x[:,:,::-1]
    x= np.clip(x,0,255).astype('uint8')
    
    return x

In [9]:
def continuity_loss(x):
    assert k.ndim(x) == 4
    a= k.square(x[:,:height-1,:width-1,:]-x[:,1:,:width-1,:])
    b= k.square(x[:,:height-1,:width-1,:]-x[:,:height-1,1:,:])
    return k.sum(k.pow(a+b,1.25))

In [10]:
def eval_loss_and_grads(x):
    x = x.reshape((1,) + image_size)
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [11]:
image_size=(height,width,3)
dream= Input(batch_shape=(1,)+image_size)

In [12]:
model= vgg16.VGG16(input_tensor=dream,weights='imagenet',include_top=False)

In [13]:
layer_dict= dict([(layer.name,layer) for layer in model.layers])
loss= k.variable(0.)
for layer_name in settings['features']:
    coeff= settings['features'][layer_name]
    x= layer_dict[layer_name].output
    shape= layer_dict[layer_name].output_shape
    loss-= coeff* k.sum(k.square(x[:,2:shape[1]-2,2:shape[2]-2,:]))/np.prod(shape[1:])
loss += settings['continuity'] * continuity_loss(dream) / np.prod(image_size)
loss+= settings['dreaml2']* k.sum(k.square(dream))/np.prod(image_size)
grads= k.gradients(loss,dream)
outputs=[loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs= k.function([dream],outputs)
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
evaluator= Evaluator()
x= preprocess_image(image_path)

In [14]:
for i in range(5):
    print('Start of iteration', i+1)
    start_time = time.time()
    random_jitter= (settings['jitter']*2)*(np.random.random(image_size)-0.5)
    x+= random_jitter
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),fprime=evaluator.grads, maxfun=8)
    print('Current loss value:', min_val)
    x= x.reshape(image_size)
    x-=random_jitter
    image= deprocess_image(np.copy(x))
    #print(image)
    fn = result_prefix + '_at_iteration_%d.png' % i
    imsave(fn, image)

    end_time = time.time()
    print('Image saved as', fn)
    print('Iteration %d completed in %ds' % (i, end_time-start_time))

Start of iteration 1
Current loss value: -207.856
Image saved as /home/harshit/Downloads_at_iteration_0.png
Iteration 0 completed in 270s
Start of iteration 2
Current loss value: -1028.04
Image saved as /home/harshit/Downloads_at_iteration_1.png
Iteration 1 completed in 246s
Start of iteration 3
Current loss value: -3487.25
Image saved as /home/harshit/Downloads_at_iteration_2.png
Iteration 2 completed in 242s
Start of iteration 4
Current loss value: -9040.77
Image saved as /home/harshit/Downloads_at_iteration_3.png
Iteration 3 completed in 241s
Start of iteration 5
Current loss value: -19578.7
Image saved as /home/harshit/Downloads_at_iteration_4.png
Iteration 4 completed in 241s
